### <center>Enviando Dados ao MinIO</center>

--------

#### Bibliotecas

In [1]:
from minio import Minio
from pathlib import Path

#### Variáveis Locais

In [2]:
# Variáveis RAW e PARQUET
PARQUET_DIR = Path("..") / "data" / "parquet"

In [3]:
# Variáveis MinIO
MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
BUCKET_NAME = "xau-lake"

#### Validando a Existência do Bucket

In [4]:
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False,
)

In [5]:
if not client.bucket_exists(BUCKET_NAME):
    raise RuntimeError(f"Bucket {BUCKET_NAME} não existe. Crie antes no 01_download_raw.")
else:
    print(f"Bucket '{BUCKET_NAME}' encontrado.")

Bucket 'xau-lake' encontrado.


#### Enviando Dados ao MinIO

In [6]:
for pq_path in PARQUET_DIR.glob("XAU_*_data.parquet"):
    # Ex.: XAU_15m_data.parquet -> ["XAU", "15m", "data"] -> "15m"
    parts = pq_path.stem.split("_")
    timeframe = parts[1]

    object_name = f"silver/timeframe={timeframe}/{pq_path.name}"

    print(f"Enviando {pq_path.name} como {object_name}...")

    client.fput_object(
        BUCKET_NAME,
        object_name,
        str(pq_path)
    )

print("Upload concluído.")

Enviando XAU_30m_data.parquet como silver/timeframe=30m/XAU_30m_data.parquet...
Enviando XAU_4h_data.parquet como silver/timeframe=4h/XAU_4h_data.parquet...
Enviando XAU_5m_data.parquet como silver/timeframe=5m/XAU_5m_data.parquet...
Enviando XAU_15m_data.parquet como silver/timeframe=15m/XAU_15m_data.parquet...
Enviando XAU_1h_data.parquet como silver/timeframe=1h/XAU_1h_data.parquet...
Enviando XAU_1d_data.parquet como silver/timeframe=1d/XAU_1d_data.parquet...
Upload concluído.


In [7]:
# Validando os dados no MinIO
for obj in client.list_objects(BUCKET_NAME, prefix="silver/", recursive=True):
    print(obj.object_name, "-", obj.size, "bytes")

silver/timeframe=15m/XAU_15m_data.parquet - 11074674 bytes
silver/timeframe=1d/XAU_1d_data.parquet - 209125 bytes
silver/timeframe=1h/XAU_1h_data.parquet - 3565450 bytes
silver/timeframe=30m/XAU_30m_data.parquet - 6195340 bytes
silver/timeframe=4h/XAU_4h_data.parquet - 1097956 bytes
silver/timeframe=5m/XAU_5m_data.parquet - 27950016 bytes
